[Table of Contents](../../index.ipynb)

# FRC Analytics with Python - Session 16
# Structured Query Language (SQL)
**Last Updated: 8 October 2021**

This session covers *Structured Query Language* (SQL). SQL is used to to create, modify, and retrieve data stored in relational databases. SQL has been in general use since the 1980s and is required knowledge for data scientists and business analysts. SQL is included in our curriculum because the Issaquah Robotics Society's scouting system uses SQL to store, retrieve, and manipulate data.

A *database* is a software program that stores information. Relational databases are a type of database that store data in tables with rows and columns. Databases often run on computer servers and provide information to other computer programs over a network. For example, when a web browser retrieves a web page, the web page is generated by a web server like Apache, Microsoft Internet Information Services (IIS), or Nginx. Web servers typically retrieve data from relational database servers when they construct a web page. MySQL, PostgreSQL, and Oracle are some of the most common relational database servers.

The IRS's scouting system uses a simple relational database called *Sqlite*. Sqlite is bundeled with Python, so if you've installed Python on your computer, you already have Sqlite.

##### If Using Google Colab
It's best if you clone the *pyclass_frc* Github repo and run this notebook from your local computer. But if you would like to run it from Google Colab, uncomment and run the line in the next cell. (*Don't delete the exclamation point at the start of the line!*) The cell will copy a Sqlite database file from the Github repository.

In [1]:
# !wget -nv https://raw.githubusercontent.com/irs1318dev/pyclass_frc/master/sessions/s17_SQL/wasno2020.sqlite3

## I. Getting a Database Connection
To use a Sqlite database, we must first import the `sqlite3` package. This package is part of the *Python Standard Library*, so there is no need to install it with Anaconda. In addition to `sqlite3`, we'll also need the `pandas` package.

In [2]:
import sqlite3

import pandas as pd

A Sqlite database is stored as a file. Our database is contained in the file *wasno2020.sqlite3*. It contains scouting data from the 2020 Pacific Northwest (PNW) district competition at Glacier Peak High School in Snohomish, WA. We can get a connection to this database by using the `sqlite3.connect()` function. 

In [3]:
# Get a connection to the database
con = sqlite3.connect("wasno2020.sqlite3")

## II. Getting Started

### A. Our First Query
The instructions that we'll give to our database are called queries. Run the next cell to see the results of our first query.

In [9]:
# Our first SQL Query
query = "SELECT * FROM teams;"

# This statement runs the query. We'll explain it later.
teams_dataframe = pd.read_sql_query(query, con, index_col = "team_id")

# Display the first six rows
teams_dataframe.head()

,team_number,team_name,city,state,region,year_founded,matches_played
team_id,,,,,,,
1,1318,Issaquah Robotics Society,Issaquah,Washington,None,2004.0,10
2,2928,Viking Robotics,Seattle,Washington,None,2009.0,10
3,2903,NeoBots,Arlington,Washington,None,2009.0,10
4,3070,Team Pronto,Seattle,Washington,None,2009.0,9
5,2930,Sonic Squirrels,Snohomish,Washington,None,2009.0,10


The first line of the code cell defines a SQL query, `SELECT * FROM teams;`and saves it in a Python variable. The next line sends the query to our *Sqlite* database and converts the results to a *Pandas* dataframe. Don't pay much attention to this line -- we'll cover it in more detail later. The final statement displays the first six records that were retrieved from the database.

The data from the *teams* table consists of rows and columns. Each row corresponds to a different robotics team, and each column represents a different attribute of a team, such as its name, city or year founded. Rows are often called *records* and the attribute represented by a column is often called a *field*.

This particular SQL query, `SELECT * FROM teams;`, requests all data from the *teams* table. The SQL query has several components.
* Our query started with the SQL keyword `SELECT`, which means we want to retrieve data from the database.
* The asterisk, `*`, means we want to return all columns. If you were reading a SQL query out load, you could say "all columns" or "all fields" instead of "asterisk".
* The phrase `FROM teams` means we want to the data to come from the *teams* table. `FROM` is a SQL keyword and `teams` is a user-created name that we gave to our database table.
* All SQL queries end with a semicolon, `;`.

### B. Choosing Columns
We can replace the asterisk in the `SELECT` statement with one or more column names. For example, we could modify the query to provide only the *team_name* and *team_number* columns.

In [10]:
# We can choose our columns
query = """SELECT team_name, team_number FROM teams;"""

# Don't worry about this statement yet
teams_dataframe = pd.read_sql_query(query, con)

# Display the first six rows
teams_dataframe.head()

,team_name,team_number
0,Issaquah Robotics Society,1318
1,Viking Robotics,2928
2,NeoBots,2903
3,Team Pronto,3070
4,Sonic Squirrels,2930


`SELECT` statements will accept one or more column names. If providing more than one column name, separate the column names with commas.

### C. Displaying Different Column Names
Using underscores to separate the words in long column names is a good practice, but it does make our tables look a bit crude.  We can use aliases to display the columns using a different name and make our table look more polished.

In [12]:
# We can choose our columns
query = """SELECT team_name AS Team,
                  team_number AS "Team Number"
             FROM teams;"""

# Don't worry about this statement yet
teams_dataframe = pd.read_sql_query(query, con)

# Display the first six rows
teams_dataframe.head()

,Team,Team Number
0,Issaquah Robotics Society,1318
1,Viking Robotics,2928
2,NeoBots,2903
3,Team Pronto,3070
4,Sonic Squirrels,2930


The `AS` keyword caused the columns to be renamed when our results were displayed. We can include spaces in our column name if we enclose the name in double quotes (but then the entire string needs to be enclosed in single or triple quotes).

### D. Filtering Results with a WHERE Clause
The following query only returns teams that were founded in 2013.

In [24]:
# We can filter the results with a WHERE clause.
query = """SELECT *
             FROM teams
            WHERE year_founded = 2013;"""

# Don't worry about this statement yet
teams_dataframe = pd.read_sql_query(query, con, index_col="team_id")

# Display the first six rows
teams_dataframe

,team_number,team_name,city,state,region,year_founded,matches_played
team_id,,,,,,,
8,4681,Murphy's law,Everett,Washington,None,2013,10
11,4682,BraveBots,Seattle,Washington,None,2013,9
18,4683,Full Metal Robotics,Marysville,Washington,None,2013,10
23,4512,BEAR bots,Everett,Washington,None,2013,9
29,4513,Circuit Breakers,Medical Lake,Washington,None,2013,10


We filtered the query results by adding a `WHERE` clause. Here are a couple key points:
* The order of clauses matters. The `WHERE` clause must come after the `FROM` clause, or else the SQL query will fail.
* Unlike Python, the equality operator, `=`, contains a single equals sign.

We can use the Boolean operators `AND`, `OR`, and `NOT` in our `WHERE` clause.

In [32]:
# We can use Boolean operators in the WHERE clause.
query = """SELECT *
             FROM teams
            WHERE year_founded <= 2012
              AND city = 'Seattle';"""

# Don't worry about this statement yet
teams_dataframe = pd.read_sql_query(query, con, index_col="team_id")

# Display the first six rows
teams_dataframe

,team_number,team_name,city,state,region,year_founded,matches_played
team_id,,,,,,,
2,2928,Viking Robotics,Seattle,Washington,None,2009,10
4,3070,Team Pronto,Seattle,Washington,None,2009,9
31,4180,Iron Riders,Seattle,Washington,None,2012,9


The query displays all teams in Seattle that were founded in 2012 or earlier. Here are a couple important things to note:
* The value *Seattle* is surrounded by single quotes (`'`) but the value *2012* does not use any quotes. This is because the *year_founded* has a numeric datatype and *city* has a text datatype. Literal text values should always be surrounded by *single* quotes. (*Sqlite* will let you use double quotes around literal strings, but other common database servers, like *Postgres*, will not. It's best get in the habit of using single quotes in this situation.)
* While SQL keywords and user-supplied identifiers are case insensitive, SQL searches often are case-sensitive. For example, if we change our the second part of our `WHERE` clause to `AND city = 'seattle'`, our search will return no results (try it for yourself!) because all occurrences of 'Seattle' in the *teams* table are capitalized. *Sqlite* queries can be forced to conduct a case-insensitive search by adding `COLLATE NOCASE` to the end of the `WHERE` clause. Also, it's possible to specify that searches should be case-insensitive when creating a database table.

### E. SQL Syntax and Style
Unlike Python, SQL is case insensitive. SQL considers the queries `SELECT * FROM teams;` and `select * frOM TEAMS;` to be identical. This means you can't have two tables where the only differences in the table names are that some characters are upper or lower case.

To make our queries easy to read and understand, we will conform to a few style rules. These style rules come from a [SQL styleguide by Simon Holywell](https://www.sqlstyle.guide/). 
* SQL keywords like `SELECT` and `FROM` will always be all uppercase.
* User-generated table and column names will always be all lowercase.
* If table or column names contain multiple words, the words will be separated by underscores.
* Longer queries will be placed on multiple lines, with the leftmost keywords right-aligned.
* Table names will be a plural noun that describes the data stored in the table.

In addition, when writing SQL queries in Python code, the mentor recommends placing SQL queries in triple-quoted strings. This practice has two advantages:
* It allows multi-line queries, which enhances readability.
* Double quotes (`"`) and single quotes (`'`) occur frequently in SQL queries. Single and double quotes can easily be placed in single-quoted strings.




## II. Tables and Schemas

### A. Getting a Table of Tables
Most relational databases contain multiple tables. We can run a special `SELECT` query to see what tables exist in a database.

In [61]:
# Get a list of tables in the database
query = """SELECT *
             FROM sqlite_schema
             WHERE type = 'table';"""

# Run the query and display the results
pd.read_sql_query(query, con)

,type,name,tbl_name,rootpage,sql
0,table,measures,measures,53,"CREATE TABLE ""measures"" (\n""index"" INTEGER,\n ..."
1,table,status,status,115,"CREATE TABLE ""status"" (\n""index"" INTEGER,\n ""..."
2,table,teams,teams,114,CREATE TABLE teams (\n team_id INTEGER PRIM...
3,table,schedule,schedule,2,"CREATE TABLE ""schedule"" (\n match_id..."


The results indicate our database contains four tables: *measures*, *schedule*, *status*, and *teams*.

*Sqlite* databases always contain a special table called *sqlite_schema*, which lists all objects contained in the database. The *sqlite_schema* table contains five columns:
1. `type`: Specifies the type of object. There are four possible values: 'table', 'index', 'view', and 'trigger'. We used a `WHERE` clause to filter the results to just tables. 
2. `name`: The name of the object. For tables, this column contains the table's name.
3. `tbl_name`: Indexes and triggers are always associated with a table. This column specifies the associated table. But for tables and views, the value in this column is always the same as the *name* column.
4. `rootpage`: We won't be using this column. But if you must know, *Sqlite* uses a datastructure called a *B-tree* (NOT the same as a binary tree) to store objects. This column contains the location of the table within the B-tree.
5. `sql`: Contains the SQL query that was used to create the table.

### B. Database Schemas
The arrangement of database tables, including the table's fields and data types, is called a schema. We can learn more about a database schema by inspecting the *sql* fields in the *sqlite_schema* table. For example, the following query retrieves the query that was used to create the *teams* table.

In [65]:
# Get a list of tables in the database
query = """SELECT sql
             FROM sqlite_schema
             WHERE type = 'table'
               AND name = 'teams';"""

# Run the query and display the results
print(con.execute(query).fetchone()[0])

CREATE TABLE teams (
    team_id INTEGER PRIMARY KEY,
    team_number TEXT NOT NULL UNIQUE,
    team_name TEXT,
    city TEXT,
    state TEXT,
    region TEXT,
    year_founded INTEGER,
    matches_played INTEGER
)


The `CREATE TABLE` query is used to create a new table in a SQL database.  Viewing the `CREATE TABLE` query is useful because it lists all of the columns and their datatypes for a specific table. For the *teams* table, we can see that all columns have a *TEXT* datatype except for *team_id*, *year_founded*, and *matches_played*. Note that the datatype of *team_number* is actually text, even though all of the values appear to be numeric. 

The important takeaway is that all columns have a specific datatype, and only data that matches the datatype can be stored in the column. We'll cover `CREATE TABLE` queries in greater detail in a later session.

#### Pragma Queries
*Sqlite* provides another method for retrieving information about a table. The following query extracts information about the *schedule* table.

In [86]:
query = """PRAGMA table_info(schedule);"""
pd.read_sql_query(query, con, index_col="cid")

,name,type,notnull,dflt_value,pk
cid,,,,,
0,match_id,INTEGER,0,None,1
1,match_date,TEXT,1,None,0
2,comp_level,TEXT,1,None,0
3,alliance,TEXT,1,None,0
4,team,TEXT,1,None,0
5,station,TEXT,1,None,0
6,last_match,INTEGER,0,None,0


Results from `PRAGMA table_info(...)` include six different columns:
* `cid`: Contains the column ID, which is a sequence of integers, starting with zero for the first column.
* `name`: The column name.
* `type`: The column data type.
* `notnull`: If 1, the column must contain information -- it cannot be empty.
* `dflt_value`: Contains the column's default value. The default value is inserted into the column when creating a new record if no value is specified by the user.
* `pk`: If 1, indicates the column is a primary key. Primary keys will be covered in a later session.

In [80]:
query = """SELECT * FROM schedule2;"""
pd.read_sql_query(query, con)

,match_id,match_date,comp_level,alliance,team,station,last_match
0,205768,2020-02-29T11:00:00,qual,001-q,red,4131,10
1,205769,2020-02-29T11:00:00,qual,001-q,red,4683,10
2,205770,2020-02-29T11:00:00,qual,001-q,red,2412,9
3,205771,2020-02-29T11:00:00,qual,001-q,blue,1318,10
4,205772,2020-02-29T11:00:00,qual,001-q,blue,4089,9
...,...,...,...,...,...,...,...
355,206123,2020-03-01T09:30:00,qual,060-q,red,4173,1
356,206124,2020-03-01T09:30:00,qual,060-q,red,4683,1
357,206125,2020-03-01T09:30:00,qual,060-q,blue,949,1
358,206126,2020-03-01T09:30:00,qual,060-q,blue,2910,1


In [70]:
query = """SELECT * FROM measures;"""
pd.read_sql_query(query, con).columns

Index(['index', 'date', 'event', 'season', 'level', 'match', 'alliance',
       'team', 'station', 'actor', 'task', 'measuretype', 'phase', 'attempt',
       'reason', 'capability', 'successes', 'attempts', 'cycle_times',
       'last_match', 'num_matches'],
      dtype='object')

In [71]:
query = """SELECT * FROM status;"""
pd.read_sql_query(query, con)

,index,event_id,match,date
0,0,25395,060-q,2020-03-01T09:30:00


In [53]:
df_measures = pd.read_sql_query("SELECT * FROM enum_measures", con)
pd.unique(df_measures.task)

array(['startingPosition_Goal', 'movedAuto', 'launchOuter',
       'pickupPowerCellsG', 'pickupPowerCellsL', 'launchInner',
       'defendedAgainst', 'launchLower', 'startingPosition_Cen',
       'climbPosition_Park', 'tempDisabled', 'disabled',
       'climbPosition_Side', 'startingPosition_Load', 'isLevel',
       'crossOpponentSector', 'pickupPowerCellsT', 'collided', 'defense',
       'passPowerCells', 'climbPosition_Center', 'pickupPowerCellsS',
       'noShow', 'fallOver'], dtype=object)

In [52]:
df_measures = pd.read_sql_query("SELECT * FROM measures", con)
pd.unique(df_measures.task)

array(['startingPosition', 'movedAuto', 'launchOuter',
       'pickupPowerCellsG', 'pickupPowerCellsL', 'launchInner',
       'defendedAgainst', 'launchLower', 'climbPosition', 'tempDisabled',
       'disabled', 'isLevel', 'crossOpponentSector', 'pickupPowerCellsT',
       'collided', 'defense', 'passPowerCells', 'pickupPowerCellsS',
       'noShow', 'fallOver'], dtype=object)

In [56]:
pd.read_sql_query("""SELECT * FROM enum_measures WHERE task = 'startingPosition_Goal';""", con)

,index,date,event,season,level,match,alliance,team,station,actor,...,measuretype,phase,attempt,reason,capability,successes,attempts,cycle_times,last_match,num_matches
0,0,2020-02-29T16:29:00,wasno,2020,qual,034-q,blue,1294,1,robot,...,enum,auto,summary,na,Goal,1,0,0,4,9
1,3,2020-02-29T15:03:00,wasno,2020,qual,024-q,red,1294,1,robot,...,enum,auto,summary,na,Goal,1,0,0,6,9
2,15,2020-02-29T17:25:00,wasno,2020,qual,042-q,blue,1294,3,robot,...,enum,auto,summary,na,Goal,1,0,0,3,9
3,21,2020-02-29T18:42:00,wasno,2020,qual,053-q,blue,1294,3,robot,...,enum,auto,summary,na,Goal,1,0,0,1,9
4,31,2020-02-29T11:18:00,wasno,2020,qual,003-q,blue,1294,2,robot,...,enum,auto,summary,na,Goal,1,0,0,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,1371,2020-02-29T16:15:00,wasno,2020,qual,032-q,red,7461,1,robot,...,enum,auto,summary,na,Goal,1,0,0,5,10
114,1457,2020-02-29T16:22:00,wasno,2020,qual,033-q,blue,8032,3,robot,...,enum,auto,summary,na,Goal,1,0,0,5,10
115,1472,2020-02-29T14:49:00,wasno,2020,qual,022-q,blue,8059,2,robot,...,enum,auto,summary,na,Goal,1,0,0,7,10
116,1525,2020-02-29T14:28:00,wasno,2020,qual,019-q,red,949,1,robot,...,enum,auto,summary,na,Goal,1,0,0,7,10


In [34]:
query = """SELECT *
             FROM sqlite_master 
            WHERE type ='table';"""
pd.read_sql_query(query, con)

,type,name,tbl_name,rootpage,sql
0,table,measures,measures,2,"CREATE TABLE ""measures"" (\n""index"" INTEGER,\n ..."
1,table,enum_measures,enum_measures,53,"CREATE TABLE ""enum_measures"" (\n""index"" INTEGE..."
2,table,schedule,schedule,105,"CREATE TABLE ""schedule"" (\n""index"" INTEGER,\n ..."
3,table,status,status,115,"CREATE TABLE ""status"" (\n""index"" INTEGER,\n ""..."
4,table,Old_Teams,Old_Teams,113,"CREATE TABLE ""Old_Teams"" (\n""id"" INTEGER,\n ""..."
5,table,teams,teams,114,CREATE TABLE teams (\n team_id INTEGER PRIM...


### A. `pandas.read_sql_query()` Function


Of course, nothing is going to happen unless we can get the database to evaluate the SQL query and follow its instructions. For our first query, we sent the query to the database using the [`read_sql_query()` function from the *Pandas* package](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html#pandas.read_sql_query). *Sqlite* executed the query and sent the requested data to *Pandas*.  *Pandas* assembled the data into a dataframe object.

We also had to pass the *Sqlite* connection object to the `read_sql_query()` function. Without the connection object, Pandas would have no clue which database we wanted to use.

### B. Database Table


The *teams* table in our relational database contains eight columns, named *id*, *name*, *long_name*, *city*, *state*, *region*, *year_founded*, and *matches_played*. The column on the far left that contains the values 0 through 5 is not from the *Sqlite* database. It is an index column created by *Pandas*. By the way, this data was saved after the first day of competition, so none of the FRC teams had completed all of their qualification matches.

In [4]:
cur = con.cursor()

In [28]:
q = "ALTER TABLE Teams RENAME TO Old_Teams;"
con.execute(q)
con.close()

In [3]:
q = """SELECT name
FROM sqlite_master 
WHERE type ='table' AND 
    name NOT LIKE 'sqlite_%';
"""
pd.read_sql(q, con)

,name
0,measures
1,enum_measures
2,schedule
3,status
4,Old_Teams
5,teams


In [33]:
q = """SELECT * FROM teams;"""
for row in cur.execute(q):
    print(row)

(1, '1318', 'Issaquah Robotics Society', 'Issaquah', 'Washington', None, 2004, 10)
(2, '2928', 'Viking Robotics', 'Seattle', 'Washington', None, 2009, 10)
(3, '2903', 'NeoBots', 'Arlington', 'Washington', None, 2009, 10)
(4, '3070', 'Team Pronto', 'Seattle', 'Washington', None, 2009, 9)
(5, '2930', 'Sonic Squirrels', 'Snohomish', 'Washington', None, 2009, 10)
(6, '1294', 'Top Gun', 'Sammamish', 'Washington', None, 2004, 9)
(7, '949', 'Wolverine Robotics', 'Bellevue', 'Washington', None, 2002, 10)
(8, '4681', "Murphy's law", 'Everett', 'Washington', None, 2013, 10)
(9, '4205', 'ROBOCUBS', 'Sedro Woolley', 'Washington', None, 2012, 10)
(10, '4911', 'CyberKnights', 'Seattle', 'Washington', None, 2014, 10)
(11, '4682', 'BraveBots', 'Seattle', 'Washington', None, 2013, 9)
(12, '4309', '4-H Botsmiths', 'Snohomish', 'Washington', None, 2012, 10)
(13, '1778', 'Chill Out', 'Mountlake Terrace', 'Washington', None, 2006, 10)
(14, '4173', ' IMVERT (Interscholastic Mount Vernon Engineering Robotics

In [30]:
dq = "DELETE FROM teams;"
cur.execute(dq)
con.commit()

In [ ]:
q = "SELECT * FROM old_teams;"
rows = cur.execute(q).fetchall()
rows

In [79]:
q = "SELECT * FROM schedule;"
cur = con.cursor()
rows = cur.execute(q)
ins_cur = con.cursor()
for row in rows:
    print(row)
    insert_q = """INSERT INTO schedule2
                  (last_match, match_id, match_date, comp_level, alliance, team, station)
                  VALUES (?, ?, ?, ?, ?, ?, ?);"""
    ins_cur.execute(insert_q, row[1:-2])
con.commit()

(0, 1, 206124, '2020-03-01T09:30:00', 'qual', '060-q', 'red', '4683', '3', 25395)
(1, 1, 206122, '2020-03-01T09:30:00', 'qual', '060-q', 'red', '3268', '1', 25395)
(2, 1, 206125, '2020-03-01T09:30:00', 'qual', '060-q', 'blue', '949', '1', 25395)
(3, 1, 206123, '2020-03-01T09:30:00', 'qual', '060-q', 'red', '4173', '2', 25395)
(4, 1, 206126, '2020-03-01T09:30:00', 'qual', '060-q', 'blue', '2910', '2', 25395)
(5, 1, 206127, '2020-03-01T09:30:00', 'qual', '060-q', 'blue', '5588', '3', 25395)
(6, 1, 206118, '2020-02-29T19:24:00', 'qual', '059-q', 'red', '1778', '3', 25395)
(7, 1, 206119, '2020-02-29T19:24:00', 'qual', '059-q', 'blue', '2930', '1', 25395)
(8, 1, 206117, '2020-02-29T19:24:00', 'qual', '059-q', 'red', '8032', '2', 25395)
(9, 1, 206116, '2020-02-29T19:24:00', 'qual', '059-q', 'red', '4918', '1', 25395)
(10, 1, 206120, '2020-02-29T19:24:00', 'qual', '059-q', 'blue', '8248', '2', 25395)
(11, 1, 206121, '2020-02-29T19:24:00', 'qual', '059-q', 'blue', '4131', '3', 25395)
(12, 1, 2

In [26]:
q = "pragma table_info(Teams);"
pd.read_sql(q, con)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,0
1,1,name,TEXT,0,None,0
2,2,long_name,TEXT,0,None,0
3,3,city,TEXT,0,None,0
4,4,state,TEXT,0,None,0
5,5,region,TEXT,0,None,0
6,6,year_founded,TEXT,0,None,0
7,7,matches_played,INTEGER,0,None,0


In [33]:
q = """
CREATE TABLE teams (
    team_id INTEGER PRIMARY KEY,
    team_number TEXT NOT NULL UNIQUE,
    long_name TEXT,
    city TEXT,
    state TEXT,
    region TEXT,
    year_founded INTEGER,
    matches_played INTEGER
);
"""
con.execute(q)
con.commit()

In [37]:
sq = "SELECT * FROM Old_Teams;"
cur = con.cursor()
cur.execute(q)

In [38]:
for tm in cur:
    print(tm)

('measures',)
('enum_measures',)
('schedule',)
('status',)
('Old_Teams',)
('teams',)


In [23]:
# Our first query
query = "SELECT * FROM Teams;"
teams = pd.read_sql(query, con)
teams.head()

,id,name,long_name,city,state,region,year_founded,matches_played
0,0,1318,Issaquah Robotics Society,Issaquah,Washington,None,2004,10
1,1,2928,Viking Robotics,Seattle,Washington,None,2009,10
2,2,2903,NeoBots,Arlington,Washington,None,2009,10
3,3,3070,Team Pronto,Seattle,Washington,None,2009,9
4,4,2930,Sonic Squirrels,Snohomish,Washington,None,2009,10


In [57]:
q = "DROP TABLE measures;"
con.execute(q)
con.commit()

Our first query returned the first six rows of the *Teams* table. This table lists all of the FRC teams that participated in the competition. Each row contains information on a different team

In [8]:
# q = "ALTER TABLE Teams DROP COLUMN id"
# con.execute(q)
# con.commit()

In [18]:
q = "pragma table_info(Teams);"
pd.read_sql(q, con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,id,INTEGER,0,None,0
2,2,name,TEXT,0,None,0
3,3,long_name,TEXT,0,None,0
4,4,city,TEXT,0,None,0
5,5,state,TEXT,0,None,0
6,6,region,TEXT,0,None,0
7,7,year_founded,TEXT,0,None,0
8,8,matches_played,INTEGER,0,None,0


In [87]:
con.close()

ALTER TABLE Locations
RENAME COLUMN Address TO Street;

In [82]:
q = """ALTER TABLE schedule2
            RENAME TO schedule;"""
con.execute(q)
con.commit()

In [74]:
q = """CREATE TABLE schedule2 (
           match_id INTEGER PRIMARY KEY,
         match_date TEXT NOT NULL,
         comp_level TEXT NOT NULL,
           alliance TEXT NOT NULL,
               team TEXT NOT NULL,
            station TEXT NOT NULL,
         last_match INTEGER);"""

con.execute(q)
con.commit()
                       
                    
                    

In [ ]:
CREATE TABLE teams (
    team_id INTEGER PRIMARY KEY,
    team_number TEXT NOT NULL UNIQUE,
    team_name TEXT,
    city TEXT,
    state TEXT,
    region TEXT,
    year_founded INTEGER,
    matches_played INTEGER
)